In [61]:
import os
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point

divide_size = 6
zone_size = int(divide_size / 2)

# 폴더 경로 (예시 경로입니다, 실제 폴더 경로를 사용자가 제공해야 함)
folder_path = r'D:\TEMP\1Tray'

# 폴더 내의 모든 CSV 파일을 찾기
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [119]:
def convert_to_numeric(column):
    return pd.to_numeric(column.str.replace(' ', ''), errors='coerce')

# CSV 파일 로딩
# data = pd.read_csv(r'D:\TEMP\1Tray\1_1.csv', names=['X', 'Y', 'Height'], header=None)
data = pd.read_csv(r'D:\TEMP\1Tray\1_1.csv', names=['X', 'Y', 'Height'], skiprows=1)
# data = data[1:]
data
# CSV 파일 로딩, 첫 번째 행을 컬럼명으로 사용, 데이터를 숫자형으로 변환
# data['X'] = convert_to_numeric(data['X'])
# data['Y'] = convert_to_numeric(data['Y'])
# data['Height'] = convert_to_numeric(data['Height'])
# data

,X,Y,Height
0,-1.00,10.90,0.018812
1,-1.00,9.91,0.020025
2,-1.99,9.91,0.018980
3,-1.99,8.92,0.019774
4,-3.47,9.41,0.017461
...,...,...,...
984,10.88,7.92,0.018774
985,10.39,5.44,0.018329
986,9.89,2.97,0.016712
987,10.88,4.95,0.017191


In [177]:
file = r'D:\TEMP\1Tray\1_1.csv'

# 파일 경로
file_path = os.path.join(folder_path, file)

# 파일 경로 설정 및 데이터 로드
data = pd.read_csv(file_path, names=['X', 'Y', 'Height'], skiprows=1)
# data = data[1:]
    
# # CSV 파일 로딩, 첫 번째 행을 컬럼명으로 사용, 데이터를 숫자형으로 변환
# data['X'] = convert_to_numeric(data['X'])
# data['Y'] = convert_to_numeric(data['Y'])
# data['Height'] = convert_to_numeric(data['Height'])


# 데이터 타입 변환
data = data.apply(pd.to_numeric, errors='coerce')

In [180]:

# 좌상단과 우하단 점 찾기
min_x, max_x = data['X'].min(), data['X'].max()
min_y, max_y = data['Y'].min(), data['Y'].max()

toler = 0.5
left_top_data = data[(data['X'] >= min_x - toler) & (data['X'] < min_x + toler) &
                            (data['Y'] >= max_y - toler) & (data['Y'] < max_y + toler)]

right_btm_data = data[(data['X'] >= max_x - toler) & (data['X'] < max_x + toler) &
                            (data['Y'] >= min_y - toler) & (data['Y'] < min_y + toler)]

right_top_data = data[(data['X'] >= max_x - toler) & (data['X'] < max_x + toler) &
                            (data['Y'] >= max_y - toler) & (data['Y'] < max_y + toler)]

left_btm_data = data[(data['X'] >= min_x - toler) & (data['X'] < min_x + toler) &
                            (data['Y'] >= min_y - toler) & (data['Y'] < min_y + toler)]

left_top_pt = left_top_data.values[0][:2]
right_btm_pt = right_btm_data.values[0][:2]
left_btm_pt = left_btm_data.values[0][:2]
right_top_pt = right_top_data.values[0][:2]

height_diff = left_btm_data.values[0][2] - left_top_data.values[0][2]


In [182]:


def AdjustSlope(data, point1, point2, height_diff):
    
    # 가상의 선 생성
    line1 = LineString([point1, point2])

    # 선에 가까운 점들 찾기
    dist_threshold = 0.5
    close_points = data[data.apply(lambda row: line1.distance(Point(row['X'], row['Y'])), axis=1) <= dist_threshold]
    
    # 높이 보정    
    x_diff = point2[0] - point1[0]
    y_diff = point2[1] - point1[1]
    slope = height_diff / np.sqrt(x_diff**2 + y_diff**2)

    # 좌상단 점으로부터의 거리 계산
    close_points['Dist'] = np.sqrt((close_points['X'] - point1[0]) ** 2 + (close_points['Y'] - point1[1]) ** 2)

    # 높이 보정 적용
    close_points['Adjusted_Height'] = close_points['Height'] + (close_points['Dist'] * slope)

    return close_points

In [186]:
close_points2 = AdjustSlope(data, left_btm_pt, right_top_pt, height_diff)

C:\Users\JINO\AppData\Local\Temp\ipykernel_19780\2806367016.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_points['Dist'] = np.sqrt((close_points['X'] - point1[0]) ** 2 + (close_points['Y'] - point1[1]) ** 2)
C:\Users\JINO\AppData\Local\Temp\ipykernel_19780\2806367016.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_points['Adjusted_Height'] = close_points['Height'] + (close_points['Dist'] * slope)


In [193]:
close_points1 = AdjustSlope(data, left_top_pt, right_btm_pt, height_diff)
close_points2 = AdjustSlope(data, left_btm_pt, right_top_pt, height_diff)

# 두 결과 합치기 (공통 열 확인 필요)
close_points = pd.concat([close_points1, close_points2]).reset_index(drop=True)


C:\Users\JINO\AppData\Local\Temp\ipykernel_19780\2806367016.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_points['Dist'] = np.sqrt((close_points['X'] - point1[0]) ** 2 + (close_points['Y'] - point1[1]) ** 2)
C:\Users\JINO\AppData\Local\Temp\ipykernel_19780\2806367016.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_points['Adjusted_Height'] = close_points['Height'] + (close_points['Dist'] * slope)
C:\Users\JINO\AppData\Local\Temp\ipykernel_19780\2806367016.py:16: SettingWithCopyWarnin

In [194]:
close_points

,X,Y,Height,Dist,Adjusted_Height
0,-1.01,0.99,0.022152,13.986579,0.027231
1,-1.99,1.99,0.022112,12.586501,0.026682
2,-10.89,10.89,0.006874,0.000000,0.006874
3,-1.50,1.49,0.021575,13.286538,0.026400
4,-7.43,7.43,0.014664,4.893179,0.016441
...,...,...,...,...,...
77,6.92,6.93,0.020479,25.208358,0.029841
78,5.41,5.43,0.021191,23.079970,0.029763
79,10.38,10.40,0.019933,30.108610,0.031115
80,9.40,9.41,0.020813,28.715607,0.031478


In [190]:
merged_data


,X,Y,Height_x,Dist_x,Adjusted_Height_x,Height_y,Dist_y,Adjusted_Height_y
0,-0.01,0.0,0.022481,15.393716,0.028071,0.022481,15.407858,0.028203


In [188]:
close_points

,X,Y,Height,Dist,Adjusted_Height


In [174]:


def AdjustSlope(data, point1, point2, height_diff):
    
    # 가상의 선 생성
    line1 = LineString([point1, point2])

    # 선에 가까운 점들 찾기
    dist_threshold = 0.5
    close_points = data[data.apply(lambda row: line1.distance(Point(row['X'], row['Y'])), axis=1) <= dist_threshold]
    
    # 높이 보정    
    x_diff = point2[0] - point1[0]
    y_diff = point2[1] - point1[1]
    slope = height_diff / np.sqrt(x_diff**2 + y_diff**2)

    # 좌상단 점으로부터의 거리 계산
    close_points['Dist'] = np.sqrt((close_points['X'] - point1[0]) ** 2 + (close_points['Y'] - point1[1]) ** 2)

    # 높이 보정 적용
    close_points['Adjusted_Height'] = close_points['Height'] + (close_points['Dist'] * slope)

    return close_points

In [166]:
close_points['Dist']

55     0.0
125    0.0
137    0.0
144    0.0
159    0.0
164    0.0
165    0.0
186    0.0
200    0.0
202    0.0
204    0.0
253    0.0
258    0.0
261    0.0
279    0.0
335    0.0
416    0.0
443    0.0
501    0.0
503    0.0
505    0.0
530    0.0
535    0.0
581    0.0
583    0.0
600    0.0
606    0.0
645    0.0
662    0.0
663    0.0
721    0.0
723    0.0
724    0.0
754    0.0
755    0.0
813    0.0
845    0.0
852    0.0
880    0.0
910    0.0
971    0.0
Name: Dist, dtype: float64

In [73]:
line1 = LineString([left_top_data.values[0][:2], right_btm_data.values[0][:2]])


In [75]:

# 가상의 선 생성
line1 = LineString([left_top_data.values[0][:2], right_btm_data.values[0][:2]])
line2 = LineString([left_btm_data.values[0][:2], right_top_data.values[0][:2]])

# 일정 거리(dist) 설정
dist = 2

# 일정 거리(dist) 내에 있는 좌표 추출
diagonal_section1 = []
diagonal_section2 = []

data_cnt = data.shape[0]

In [111]:

left_top_pt = left_top_data.values[0][:2]
right_btm_pt = right_btm_data.values[0][:2]
left_btm_pt = left_btm_data.values[0][:2]
right_top_pt = right_top_data.values[0][:2]


# 가상의 선 방향 계산
direction_vector1 = right_btm_pt - left_top_pt
direction_vector1 /= np.linalg.norm(direction_vector1)  # 방향 벡터 정규화

direction_vector2 = right_top_pt - left_btm_pt
direction_vector2 /= np.linalg.norm(direction_vector2)  # 방향 벡터 정규화


diagonal_section1 = np.array([])
diagonal_section2 = np.array([])

for index, row in data.iterrows():        
    point = Point(row['X'], row['Y'])
    if line1.distance(point) <= dist:
        continue

    point = np.array(row['X'], row['Y'])
    # if np.allclose(point[:2], left_top_pt) or np.allclose(point[:2], right_btm_pt):
    #     continue  # 좌상과 우하 좌표는 보정할 필요 없음

    height_difference1 = np.dot(point - left_top_pt, direction_vector1)  # 가상의 선과 수직 거리 계산
    height_difference2 = np.dot(point - left_btm_pt, direction_vector2)  # 가상의 선과 수직 거리 계산
   
    distance1 = np.linalg.norm(point - left_top_pt)  # 좌상단과의 거리
    distance2 = np.linalg.norm(point - left_btm_pt)  # 좌하단과의 거리

    diagonal_section1.append((distance1, row['Height'] + height_difference1))
    diagonal_section2.append((distance2, row['Height'] + height_difference2))
    

    height_difference = np.dot(point - left_top_pt, direction_vector1) * (right_btm_pt - left_top_pt)
    print('height_difference = ',height_difference)
    print(height_difference,row['Height'])
    adjust_height = row['Height'] + height_difference        
    distance = np.linalg.norm(point - left_top_pt)        
    print(distance)
    diagonal_section1 = np.array(diagonal_section1,(distance,adjust_height))


height_difference =  [ 335.28216517 -335.43617627]
[ 335.28216517 -335.43617627] 0.018812
15.465581140067128


ValueError: invalid shape in fixed-type tuple.

In [95]:
len(diagonal_section1)

122

In [83]:
diagonal_section1

[X        -1.500000
 Y        -0.490000
 Height    0.021857
 Name: 45, dtype: float64,
 X        -1.010000
 Y         0.990000
 Height    0.022152
 Name: 56, dtype: float64,
 X        -0.990000
 Y        -0.990000
 Height    0.022651
 Name: 57, dtype: float64,
 X        -2.980000
 Y         4.950000
 Height    0.021144
 Name: 71, dtype: float64,
 X        -1.500000
 Y         3.470000
 Height    0.021947
 Name: 72, dtype: float64,
 X        -1.010000
 Y         1.980000
 Height    0.022193
 Name: 73, dtype: float64,
 X        -2.490000
 Y         1.490000
 Height    0.021752
 Name: 74, dtype: float64,
 X        -1.500000
 Y         0.500000
 Height    0.021973
 Name: 75, dtype: float64,
 X        -2.490000
 Y         0.500000
 Height    0.021927
 Name: 76, dtype: float64,
 X        -2.000000
 Y         0.010000
 Height    0.022087
 Name: 77, dtype: float64,
 X        -3.970000
 Y         5.950000
 Height    0.019523
 Name: 105, dtype: float64,
 X        -1.990000
 Y         3.970000
 H

In [43]:


data.loc[(data['X'] == data['X'].min()) & (data['Y'] == data['Y'].max())]

,X,Y,Height


In [41]:
left_top_index = data.loc[(data['X'] == data['X'].min()) & (data['Y'] == data['Y'].max())].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [38]:

# 좌상 끝점과 우하 끝점 및 높이 구하기
left_top = np.array([data['X'].min(), data['Y'].max(), data.loc[data['X'].idxmin(), 'Height']])
right_bottom = np.array([data['X'].max(), data['Y'].min(), data.loc[data['X'].idxmax(), 'Height']])


left_top_index = data.loc[(data['X'] == data['x'].min()) & (data['Y'] == data['Y'].max())].index[0]


min_x = data['X'].min()
max_x = data['X'].max()
min_y = data['Y'].min()
max_y = data['Y'].max()

min_x_y = data[data['X'] == min_x]['Y'].max()
right_top = data[data['X'] == max_x]['Y'].max()
right_btm = data[data['X'] == max_x]['Y'].min()
left_btm = data[data['X'] == min_x]['Y'].min()

left_top_height = data[(data['X'] == min_x) & (data['Y'] == min_x_y)]['Height'].values[0]




In [39]:
left_top

9.9

In [48]:
# 데이터를 숫자형으로 변환하는 함수
def convert_to_numeric(column):
    return pd.to_numeric(column.str.replace(' ', ''), errors='coerce')

def AverageSection(file):

    print('file :', file)
    
    # 파일 경로
    file_path = os.path.join(folder_path, file)
    
    # CSV 파일 로딩
    data = pd.read_csv(file_path, names=['X', 'Y', 'Height'], header=None)
    data = data[1:]
        
    # CSV 파일 로딩, 첫 번째 행을 컬럼명으로 사용, 데이터를 숫자형으로 변환
    data['X'] = convert_to_numeric(data['X'])
    data['Y'] = convert_to_numeric(data['Y'])
    data['Height'] = convert_to_numeric(data['Height'])

    min_x, max_x = data['X'].min(), data['X'].max()
    min_y, max_y = data['Y'].min(), data['Y'].max()

    # 영역을 6등분할 크기를 계산
    x_interval = (max_x - min_x) / divide_size
    y_interval = (max_y - min_y) / divide_size

    section_avg = np.zeros(shape=(divide_size,divide_size), dtype=float)

    # 6x6 영역을 순회하며 평균 높이 계산
    for i in range(divide_size):
        for j in range(divide_size):
            # 현재 영역의 X, Y 좌표 범위를 계산
            x_start = min_x + i * x_interval
            x_end = x_start + x_interval
            y_start = min_y + j * y_interval
            y_end = y_start + y_interval
            
            # 현재 영역에 해당하는 데이터를 필터링
            area_data = data[(data['X'] >= x_start) & (data['X'] < x_end) &
                             (data['Y'] >= y_start) & (data['Y'] < y_end)]
            
            # 영역별 평균 높이를 계산하여 리스트에 추가
            avg_height = area_data['Height'].mean() if not area_data.empty else None
            section_avg[i,j] = avg_height
    return section_avg

In [53]:
def AverageZone(section_avg):

    #0 : 중심, 1 : 좌상, 2:우상, 3:좌하, 4:우하
    zone_cnt = 5
    zone_avg = np.zeros(shape=(5), dtype=float)

    for k in range(zone_cnt):
        
        x_start = 0
        y_start = 0
        x_size = zone_size
        y_size = zone_size

        if k == 0:
            x_start = 2
            y_start = 2
            x_size = 2
            y_size = 2
        elif k == 1:
            x_start = 0
            y_start = 0
        elif k == 2:
            x_start = 3
            y_start = 0
        elif k == 3:
            x_start = 0
            y_start = 3
        elif k == 4:
            x_start = 3
            y_start = 3

        total_val = 0
        add_cnt = 0

        # 3x3 영역을 순회하며 평균 높이 계산
        for i in range(x_size):
            for j in range(y_size):
                
                if k > 0:
                    # 중심이 아닌 Zone을 구할때, 중심 영역일때 스킵
                    if((i == 2 or i == 3) and (j == 2 or j== 3)):
                       continue
                
                total_val += section_avg[i,j]
                add_cnt += 1
        
        avg_val = total_val / add_cnt
        zone_avg[k] = avg_val

    return zone_avg